Group by age and generation. Join with replicate weights and calculate standard errors.

In [1]:
import pandas as pd
from math import sqrt

from IPython.display import display, HTML

Set options

In [2]:
pd.options.display.float_format = '{:,.0f}'.format

Function for building table of estimates and standard errors

In [3]:
def calc_estimates(group):
    
    # Constant from Formula (16) in the SIPP Source and Accuracy Statement
    # 240 = number of replicant columns
    const = (1 / (240 * (0.5**2)))
    
    cweight = group['wpfinwgt'].sum()
    # get only the rep columns
    cols = [ each for each in group.columns if each[0:5] == "repwt"]
    
    # sum each colummn
    sums = group[cols].apply(lambda col: col.sum())
    
    res = sums.apply( lambda each: (each - cweight)**2 )
    var = const * res.sum()
    stder = sqrt(var)
    conf = stder * 1.645
    return pd.Series([cweight, conf, stder])


Function for grouping by age

In [4]:
def group_age(joined, variable):
    
    df = pd.DataFrame(
        joined.groupby(variable).apply(calc_estimates)
    ).reset_index().rename( columns = {
        0: "estimate",
        1: "conf_interval",
        2: "standard_error",
    })
    
    return df
    

Function for grouping and joining for both variables

In [5]:
# takes a df of unique supporters and the replicates file
# returns a dataframe containing age group estimates and standard errors
def group_and_join(supports, replicates):
    
    joined = (
        supports
        .merge(
            replicates,
            on = "uid",
            how = "left"
        )
    )
    
    ages = group_age(joined, 'age_group')
    
    generations = group_age(joined, 'generation')
    
    return ages, generations
    

Run for each in Wave 1

In [6]:
wave1_files = ["../output/w1_supports_children.csv", "../output/w1_supports_parents.csv"]
wave1_replicates = pd.read_csv("../output/w1_replicates.csv")

for each in wave1_files:
    print(each.split("/")[-1])
    
    supports = pd.read_csv(each, dtype = {"uid": "object"})
    
    dfs = group_and_join(supports, wave1_replicates)
    
    [display(df) for df in dfs]
    

w1_supports_children.csv


,age_group,estimate,conf_interval,standard_error
0,30-39,"145,244","49,761","30,250"
1,40-49,"565,397","100,725","61,231"
2,50-59,"910,687","122,259","74,321"
3,60 years and over,"924,824","124,207","75,506"
4,under 30,"52,046","26,253","15,959"


,generation,estimate,conf_interval,standard_error
0,Boomer,"1,527,262","154,016","93,627"
1,Gen Z,"1,508","2,490","1,514"
2,Generation x,"607,156","97,983","59,564"
3,Millennials,"101,292","38,155","23,195"
4,Silent,"360,980","79,200","48,146"


w1_supports_parents.csv


,age_group,estimate,conf_interval,standard_error
0,30-39,"1,208,062","142,681","86,736"
1,40-49,"1,029,450","141,781","86,189"
2,50-59,"784,984","109,440","66,529"
3,60 years and over,"264,215","60,542","36,804"
4,under 30,"1,102,351","159,549","96,990"


,generation,estimate,conf_interval,standard_error
0,Boomer,"1,063,121","129,048","78,448"
1,Gen Z,"108,070","39,523","24,026"
2,Generation x,"1,813,100","194,923","118,494"
3,Millennials,"1,362,416","169,199","102,856"
4,Silent,"42,355","26,853","16,324"


Run for each in Wave 4

In [7]:
wave4_files = ["../output/w4_supports_children.csv", "../output/w4_supports_parents.csv"]
wave4_replicates = pd.read_csv("../output/w4_replicates.csv")

for each in wave4_files:
    print(each.split("/")[-1])
    
    supports = pd.read_csv(each, dtype = {"uid": "object"})
        
    dfs = group_and_join(supports, wave4_replicates)
    
    [display(df) for df in dfs]
    

w4_supports_children.csv


,age_group,estimate,conf_interval,standard_error
0,30-39,"145,244","94,582","57,497"
1,40-49,"565,397","231,598","140,789"
2,50-59,"910,687","1,014,765","616,878"
3,60 years and over,"924,824","208,240","126,590"
4,under 30,"52,046","37,378","22,722"


,generation,estimate,conf_interval,standard_error
0,Boomer,"1,527,262","1,087,168","660,892"
1,Gen Z,"1,508","8,970","5,453"
2,Generation x,"607,156","238,693","145,102"
3,Millennials,"101,292","59,185","35,979"
4,Silent,"360,980","252,808","153,683"


w4_supports_parents.csv


,age_group,estimate,conf_interval,standard_error
0,30-39,"1,208,062","849,805","516,599"
1,40-49,"1,029,450","650,319","395,330"
2,50-59,"784,984","514,167","312,563"
3,60 years and over,"264,215","201,982","122,786"
4,under 30,"1,102,351","624,566","379,675"


,generation,estimate,conf_interval,standard_error
0,Boomer,"1,063,121","361,194","219,571"
1,Gen Z,"108,070","130,794","79,510"
2,Generation x,"1,813,100","1,168,401","710,274"
3,Millennials,"1,362,416","296,563","180,282"
4,Silent,"42,355","41,064","24,963"


## Hypothesis test

Testing this statement: "In 2016, more Baby Boomers provided for children outside the home than Millennials provided for parents living outside the home."

In [8]:
w4_parents_main = pd.read_csv(
    "../output/w4_supports_parents.csv", 
    dtype = {"uid": "object"}
)

w4_children_main = pd.read_csv(
    "../output/w4_supports_children.csv", 
    dtype = {"uid": "object"}
)

w4_par = group_and_join(w4_parents_main, wave4_replicates)[1]
w4_chi = group_and_join(w4_children_main, wave4_replicates)[1]

In [9]:
w4_par

,generation,estimate,conf_interval,standard_error
0,Boomer,"1,063,121","361,194","219,571"
1,Gen Z,"108,070","130,794","79,510"
2,Generation x,"1,813,100","1,168,401","710,274"
3,Millennials,"1,362,416","296,563","180,282"
4,Silent,"42,355","41,064","24,963"


In [10]:
w4_chi

,generation,estimate,conf_interval,standard_error
0,Boomer,"1,527,262","1,087,168","660,892"
1,Gen Z,"1,508","8,970","5,453"
2,Generation x,"607,156","238,693","145,102"
3,Millennials,"101,292","59,185","35,979"
4,Silent,"360,980","252,808","153,683"


In [11]:
boomer = w4_chi.loc[ lambda x: x['generation'] == "Boomer"]
millennial = w4_par.loc[ lambda x: x['generation'] == "Millennials"]

In [12]:
# calculate standard error of a difference
def get_sdiff(a,b):
    sdiff = sqrt(a**2 + b**2)
    
    return sdiff

In [13]:
# test standard error
def test(a, b):
    sdiff = get_sdiff(a['standard_error'].array[0], b['standard_error'].array[0])
    
    diff = a["estimate"].array[0] - b["estimate"].array[0]
    
    return ~((-1.645 * sdiff) < diff < (1.645 * sdiff))

In [14]:
test(boomer, millennial)

False

---
---
---